# Semantic Inpainting using DCGANs
* * *

This is an experimental tensorflow implementation of semantic inpainting from corrupted images using DCGANs from the paper [Semantic Image Inpainting with Perceptual and Contextual Losses](https://arxiv.org/abs/1607.07539). A major help was Brandon Amos blog on [Image Completion](https://bamos.github.io/2016/08/09/deep-completion/).One of the major difference between is the training method used.I have used Adam Optimizer instead of gradient descent.
* * * *
## Requirements
* Tensorflow
* glob
* Python 3
* * *
## Dataset
* I have used Celebrity faces dataset [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html).Download the aligned version ,extract in the same directory as the code.


In [1]:
import os
print(os.listdir())

['.git', '.ipynb_checkpoints', 'Image Completion (2).ipynb', 'Image Completion.ipynb', 'images', 'image_helpers.py', 'image_helpers.pyc', 'img_align_celeba', 'ops.py', 'ops.pyc', 'README.md', '__pycache__']


In [2]:
import numpy as np
import tensorflow as tf
import time
from glob import glob

from ops import batch_norm,linear,conv2d,deconv2d,lrelu
from image_helpers import *

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Model Defintion

![alt text](images/model.png)

In [3]:
#Parameter Defintion
is_crop=True
batch_size=64
image_size=108
sample_size=64
image_shape=[64,64,3]

z_dim=100

gf_dim=64
df_dim=64

learning_rate=0.0002
beta1=0.5

In [4]:
#Batch Normalisation objects
d_bn1 = batch_norm(name='d_bn1')
d_bn2 = batch_norm(name='d_bn2')
d_bn3 = batch_norm(name='d_bn3')

g_bn0 = batch_norm(name='g_bn0')
g_bn1 = batch_norm(name='g_bn1')
g_bn2 = batch_norm(name='g_bn2')
g_bn3 = batch_norm(name='g_bn3')

In [5]:
def discriminator(image,reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
        
    h0=lrelu(conv2d(image,df_dim,name='d_h0_conv'))
    h1=lrelu(d_bn1(conv2d(h0,df_dim*2,name='d_h1_conv')))
    h2=lrelu(d_bn2(conv2d(h1,df_dim*4,name='d_h2_conv')))
    h3=lrelu(d_bn3(conv2d(h2,df_dim*8,name='d_h3_conv')))
    h4=linear(tf.reshape(h3,[batch_size,-1]),1,'d_h3_lin')
             
    return tf.nn.sigmoid(h4),h4

In [6]:
def generator(z):
    z_=linear(z,gf_dim*8*4*4,'g_h0_lin')
    h0=tf.nn.relu(g_bn0(tf.reshape(z_,[-1,4,4,gf_dim*8])))
    h1=tf.nn.relu(g_bn1(deconv2d(h0,[batch_size,8,8,gf_dim*4],name='g_h1')))
    h2=tf.nn.relu(g_bn2(deconv2d(h1,[batch_size,16,16,gf_dim*2],name='g_h2')))
    h3=tf.nn.relu(g_bn3(deconv2d(h2,[batch_size,32,32,gf_dim*1],name='g_h3')))
    h4 = deconv2d(h3, [batch_size, 64, 64, 3], name='g_h4')
    
    return tf.nn.tanh(h4)

In [7]:
#Building model
images=tf.placeholder(tf.float32,[batch_size]+image_shape,name='real_images')
sample_images=tf.placeholder(tf.float32,[sample_size]+image_shape,name="sample_images")
z=tf.placeholder(tf.float32,[None,z_dim])

G=generator(z)
D,D_logits=discriminator(images)
D_,D_logits_=discriminator(G,reuse=True)

#cost fn
d_loss_real=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits,labels=tf.ones_like(D)))
d_loss_fake=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_,labels=tf.zeros_like(D_)))
d_loss=d_loss_real+d_loss_fake

g_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logits_,labels=tf.ones_like(D_)))
#For image completion
mask=tf.placeholder(tf.float32,[None]+image_shape,name="mask")

contextual_loss=tf.reduce_sum(tf.contrib.layers.flatten(tf.abs(tf.multiply(mask,G)-tf.multiply(mask,images))))
perceptual_loss=g_loss

complete_loss=contextual_loss+perceptual_loss

In [8]:
#Optimizers
t_vars=tf.trainable_variables()

d_vars=[var for var in t_vars if 'd_' in var.name]
g_vars=[var for var in t_vars if 'g_' in var.name]
with tf.variable_scope(tf.get_variable_scope(),reuse=tf.AUTO_REUSE):
    d_optim=tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(d_loss,var_list=d_vars)
    g_optim=tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(g_loss,var_list=g_vars)
    complete_optim=tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(complete_loss,var_list=g_vars)

## Data Points

In [9]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

saver=tf.train.Saver()

In [10]:
data=glob(os.path.join('img_align_celeba/','*.jpg'))

sample_z=np.random.uniform(-1,1,size=(sample_size,z_dim))
sample_files=data[0:sample_size]
sample=[get_image(sample_file,image_size,is_crop) for sample_file in sample_files]
sample_images=np.reshape(np.array(sample).astype(np.float32),[sample_size]+image_shape)

## Training the model

In [11]:
#Training
counter=1
start_time=time.time()
for epoch in range(1):
    np.random.shuffle(data)
    batchidxs=int(len(data)/batch_size)
    
    for idx in range(batchidxs):
        #try:
            batch_files=data[idx*batch_size:(idx+1)*batch_size]
            batch=[get_image(batch_file,image_size,is_crop=is_crop) for batch_file in batch_files]
            batch_images=np.reshape(np.array(batch).astype(np.float32),[batch_size]+image_shape)

            batch_z=np.random.uniform(-1,1,[batch_size,z_dim]).astype(np.float32)

            #mask
            scale=0.25
            mask_=np.ones([batch_size]+image_shape).astype(np.float32)
            l=int(64*scale)
            u=int(64*(1.0-scale))
            mask_[:,l:u,l:u,:]=0.0
            #inverse mask
            scale=0.25
            imask_=np.zeros([batch_size]+image_shape).astype(np.float32)
            l=int(64*scale)
            u=int(64*(1.0-scale))
            imask_[:,l:u,l:u,:]=1.0
            
            
            fd={z:batch_z,images:batch_images}
            sess.run([g_optim,d_optim],feed_dict=fd)
            sess.run([complete_optim],feed_dict={z:batch_z,images:batch_images,mask:mask_})
            c_loss,dloss,gloss=sess.run([complete_loss,d_loss,g_loss],feed_dict={z:batch_z,images:batch_images,mask:mask_})
            print(counter,c_loss,dloss,gloss)
            
            if np.mod(counter,5)==0:
                sample_generated,dl,gl=sess.run([G,d_loss,g_loss],feed_dict={z:sample_z,images:sample_images})
                original_part=np.multiply(sample_images,mask_)
                generated_part=np.multiply(sample_generated,imask_)
                total=np.add(original_part,generated_part)
                save_images(total,'samples\\')
                print('[Sample] d_loss: %.8f, g_loss: %.8f' % (dl, gl))
            counter+=1  
        #except:
         #   continue

KeyboardInterrupt: 

## Saving the model

In [12]:
saver.save(sess,"checkpoint\\Model.cpkt")

'checkpoint\\Model.cpkt'

In [13]:
saver.restore(sess, "checkpoint\\Model.cpkt")
print(sess.run(tf.all_variables()))

INFO:tensorflow:Restoring parameters from checkpoint\Model.cpkt
Instructions for updating:
Please use tf.global_variables instead.
[array([[ 0.00745633, -0.0112307 ,  0.02505956, ...,  0.03560246,
         0.02141999,  0.01965306],
       [ 0.03190712,  0.01636276, -0.01691024, ..., -0.01345505,
         0.01398254,  0.00745153],
       [ 0.01234801,  0.02889191,  0.00337181, ..., -0.01626057,
        -0.02950989, -0.01385201],
       ...,
       [-0.02385918,  0.02187427,  0.01555686, ..., -0.019919  ,
        -0.04447014, -0.00099382],
       [ 0.01022242, -0.01712457,  0.00932695, ...,  0.00820829,
         0.04952683,  0.01282469],
       [-0.00826264,  0.00431652,  0.01281456, ...,  0.02583813,
         0.01573544, -0.00937193]], dtype=float32), array([-0.0002    , -0.00019999, -0.00019999, ...,  0.0002    ,
       -0.0002    , -0.00019998], dtype=float32), array([ 0.00019999,  0.0002    ,  0.0002    , -0.00019996,  0.00019999,
       -0.00019999,  0.0002    , -0.0002    ,  0.0001